# From Molecule to Manufacturing: Process Engineering with MolBuilder

MolBuilder goes beyond retrosynthesis -- it covers the full pipeline from target molecule
to production-ready process engineering: reactor selection, condition optimization,
safety assessment (GHS), cost estimation, and batch-vs-continuous scale-up analysis.

This is what makes MolBuilder unique: no other open-source Python package offers
vertically integrated process chemistry in a single `pip install`.

In [ ]:
# pip install molbuilder
from molbuilder.smiles import parse
from molbuilder.reactions.retrosynthesis import retrosynthesis
from molbuilder.reactions.synthesis_route import extract_best_route, format_route

## Step 1: Plan the synthesis

In [ ]:
mol = parse("CC(=O)OCC")  # ethyl acetate
tree = retrosynthesis(mol, max_depth=5, beam_width=5)
route = extract_best_route(tree)
print(format_route(route))

## Step 2: Select a reactor

Based on the reaction type and production scale, MolBuilder recommends
batch, CSTR, PFR, or microreactor with sizing and materials:

In [ ]:
from molbuilder.process.reactor import select_reactor

scale_kg = 100.0

for step in route.steps:
    reactor = select_reactor(step.template, scale_kg)
    print(f"Step {step.step_number}: {step.template.name}")
    print(f"  Reactor type:     {reactor.reactor_type.name}")
    print(f"  Volume:           {reactor.volume_L:.0f} L")
    print(f"  Temperature:      {reactor.temperature_C} C")
    print(f"  Pressure:         {reactor.pressure_atm} atm")
    print(f"  Mixing:           {reactor.mixing_type}")
    print(f"  Heat transfer:    {reactor.heat_transfer}")
    print(f"  Material:         {reactor.material}")
    print(f"  Equipment cost:   ${reactor.estimated_cost_usd:,.0f}")
    print()

## Step 3: Optimize conditions

In [ ]:
from molbuilder.process.conditions import optimize_conditions

for step in route.steps:
    cond = optimize_conditions(step.template, scale_kg)
    print(f"Step {step.step_number}: {step.template.name}")
    print(f"  Temperature:   {cond.temperature_C} C")
    print(f"  Pressure:      {cond.pressure_atm} atm")
    print(f"  Solvent:       {cond.solvent}")
    print(f"  Concentration: {cond.concentration_M} M")
    print(f"  Addition:      {cond.addition_rate}")
    print(f"  Time:          {cond.reaction_time_hours} h")
    print(f"  Atmosphere:    {cond.atmosphere}")
    print()

## Step 4: Safety assessment (GHS)

Automated hazard identification with 69 GHS codes, PPE requirements,
engineering controls, and emergency procedures:

In [ ]:
from molbuilder.process.safety import assess_safety

safety = assess_safety(route.steps)
for a in safety:
    print(f"Step {a.step_number}: {a.step_name}")
    print(f"  Risk level: {a.risk_level}")
    print(f"  PPE: {', '.join(a.ppe_required)}")
    print(f"  Controls: {', '.join(a.engineering_controls)}")
    print(f"  Waste: {a.waste_classification}")
    if a.hazards:
        for h in a.hazards:
            print(f"  Reagent '{h.reagent_name}': {', '.join(h.ghs_hazards)}")
    print()

## Step 5: Cost estimation

Bottom-up cost model: raw materials, labor, equipment, energy, waste disposal, overhead:

In [ ]:
from molbuilder.process.costing import estimate_cost

cost = estimate_cost(route.steps, scale_kg=scale_kg)
print(f"=== Cost Estimate ({scale_kg} kg) ===")
print(f"Total:          ${cost.total_usd:>10,.0f}")
print(f"Per kg:         ${cost.per_kg_usd:>10,.0f}")
print(f"")
print(f"  Materials:    ${cost.breakdown.raw_materials_usd:>10,.0f}")
print(f"  Labor:        ${cost.breakdown.labor_usd:>10,.0f}")
print(f"  Equipment:    ${cost.breakdown.equipment_usd:>10,.0f}")
print(f"  Energy:       ${cost.breakdown.energy_usd:>10,.0f}")
print(f"  Waste:        ${cost.breakdown.waste_disposal_usd:>10,.0f}")
print(f"  Overhead:     ${cost.breakdown.overhead_usd:>10,.0f}")

## Step 6: Scale-up analysis

Batch sizing, annual capacity, capital cost, and operating cost projections:

In [ ]:
from molbuilder.process.scale_up import analyze_scale_up

su = analyze_scale_up(route.steps, target_annual_kg=10000.0)
print(f"=== Scale-Up Analysis (10,000 kg/yr) ===")
print(f"Mode:              {su.recommended_mode}")
print(f"Batch size:        {su.batch_size_kg} kg")
print(f"Batches/year:      {su.batches_per_year}")
print(f"Cycle time:        {su.cycle_time_hours} h")
print(f"Annual capacity:   {su.annual_capacity_kg:,.0f} kg")
print(f"Capital cost:      ${su.capital_cost_usd:,.0f}")
print(f"Operating cost:    ${su.operating_cost_annual_usd:,.0f}/yr")
print(f"")
print("Risks:")
for r in su.scale_up_risks:
    print(f"  - {r}")
print("Recommendations:")
for r in su.recommendations:
    print(f"  - {r}")

## The full pipeline in one block

Here's everything together -- from SMILES to manufacturing plan:

In [ ]:
from molbuilder.smiles import parse
from molbuilder.reactions.retrosynthesis import retrosynthesis
from molbuilder.reactions.synthesis_route import extract_best_route
from molbuilder.process.reactor import select_reactor
from molbuilder.process.conditions import optimize_conditions
from molbuilder.process.safety import assess_safety
from molbuilder.process.costing import estimate_cost
from molbuilder.process.scale_up import analyze_scale_up

SMILES = "CC(=O)OCC"  # <-- replace with your target
SCALE_KG = 100.0

mol = parse(SMILES)
tree = retrosynthesis(mol)
route = extract_best_route(tree)

print(f"Target: {SMILES}")
print(f"Steps: {route.total_steps}, Overall yield: {route.overall_yield:.1f}%")
print()

for step in route.steps:
    t = step.template
    r = select_reactor(t, SCALE_KG)
    c = optimize_conditions(t, SCALE_KG)
    print(f"Step {step.step_number}: {t.name}")
    print(f"  {r.reactor_type.name} ({r.volume_L:.0f} L) | {c.temperature_C} C | {c.solvent} | {c.atmosphere}")

cost = estimate_cost(route.steps, SCALE_KG)
print(f"\nCost: ${cost.total_usd:,.0f} total (${cost.per_kg_usd:,.0f}/kg)")

su = analyze_scale_up(route.steps, SCALE_KG * 100)
print(f"Scale-up: {su.recommended_mode}, ${su.capital_cost_usd:,.0f} capex")

---

**MolBuilder** -- the only open-source Python package that goes from SMILES to manufacturing.

- PyPI: `pip install molbuilder`
- GitHub: [github.com/Taylor-C-Powell/Molecule_Builder](https://github.com/Taylor-C-Powell/Molecule_Builder)
- API: [molbuilder-api.up.railway.app](https://molbuilder-api.up.railway.app)